# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV into DataFrame

In [ ]:
# Load the data
file_path = Path('../Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

# Split the Data into Training and Testing

In [ ]:
# Create our features
X = X = df.drop(columns=["loan_status"])
X=pd.get_dummies(X)
X

In [ ]:
# Create our target
y = df["loan_status"]#values.reshape(-1, 1)
y[:5]

In [ ]:
# Check the balance of our target values
y.value_counts()

In [ ]:
df_binary_encoded=pd.get_dummies(df,columns=["homeowner","loan_status"])
df_binary_encoded

In [ ]:
X.describe()

In [48]:
# Create X_train, X_test, y_train, y_test
# YOUR CODE HERE

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [49]:
# Create the StandardScaler instance
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(58152, 10)

## Data Pre-Processing
Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [50]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [51]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler=scaler.fit(X_train)

In [52]:
# Scale the training and testing data
X_train_scaled=X_scaler.transform(X_train)
X_test_scaled=X_scaler.transform(X_test)
X_train_scaled

array([[ 0.14045627,  0.1230916 ,  0.12859477, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-0.28969527, -0.27393966, -0.27772578, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-0.48087373, -0.47526712, -0.48088605, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       ...,
       [-1.48456066, -1.46728289, -1.47278621, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-0.28969527, -0.26831599, -0.26577518, ...,  1.00731801,
        -0.81861018, -0.3377677 ],
       [-1.05440912, -1.05787955, -1.05451506, ...,  1.00731801,
        -0.81861018, -0.3377677 ]])

# Simple Logistic Regression

In [53]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train, y_train)

LogisticRegression(random_state=1)

In [54]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9543211898288821

In [55]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  571,    54],
       [   93, 18666]], dtype=int64)

In [56]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.91      1.00      0.89      0.95      0.90       625
   low_risk       1.00      1.00      0.91      1.00      0.95      0.92     18759

avg / total       0.99      0.99      0.92      0.99      0.95      0.92     19384



# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [57]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled_1, y_resampled_1 = ros.fit_resample(X_train, y_train)


# View the count of target classes with Counter
Counter(y_resampled_1)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [58]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model_2 = LogisticRegression(solver='lbfgs', random_state=1)
model_2.fit(X_resampled_1, y_resampled_1)

LogisticRegression(random_state=1)

In [59]:
y_pred_2 = model_2.predict(X_test)

In [60]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred_2)

0.9947746894823818

In [61]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix


confusion_matrix(y_test, y_pred_2)

array([[  622,     3],
       [  106, 18653]], dtype=int64)

In [62]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



### SMOTE Oversampling

In [63]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled_2,y_resampled_2=SMOTE(random_state=1,sampling_strategy=1.0).fit_resample(
   X_train,y_train
)
# View the count of target classes with Counter
Counter(y_resampled_2)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [64]:
# Train the Logistic Regression model using the resampled data
model_3 = LogisticRegression(solver='lbfgs', random_state=1)
model_3.fit(X_resampled_2, y_resampled_2)

LogisticRegression(random_state=1)

In [65]:
# Calculated the balanced accuracy score
y_pred_3 = model_3.predict(X_test)
balanced_accuracy_score(y_test, y_pred_3)

0.9948279972279972

In [66]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_3)

array([[  622,     3],
       [  104, 18655]], dtype=int64)

In [67]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_3))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       1.00      0.99      1.00      0.99      0.99      0.99     19384



# Undersampling

In this section, you will test an undersampling algorithm to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)

X_resampled_3, y_resampled_3 = cc.fit_resample(X_train, y_train)
# View the count of target classes with Counter
Counter(y_resampled_3)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_4 = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled_3, y_resampled_3)

In [ ]:
# Calculate the balanced accuracy score
y_pred_4 = model_4.predict(X_test)
balanced_accuracy_score(y_test, y_pred_4)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_4)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_4))

# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Display the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [ ]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm=SMOOTEEN(random_state=1)
X_resampled_4,y_resampled_4=sm.fit_resample(X_train,y_train)

# View the count of target classes with Counter
Counter(y_resampled_4)

In [ ]:
# Train the Logistic Regression model using the resampled data
model_5 = LogisticRegression(solver='lbfgs', random_state=1)
model_5.fit(X_resampled_4, y_resampled_4)

In [ ]:
# Calculate the balanced accuracy score
y_pred_4 = model_5.predict(X_test)
balanced_accuracy_score(y_test, y_pred_4)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_4)

In [ ]:
# Print the imbalanced classification report
rom imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred_4))

# Final Questions

1. Which model had the best balanced accuracy score?

   YOUR ANSWER HERE.

2. Which model had the best recall score?

    YOUR ANSWER HERE.

3. Which model had the best geometric mean score?

    YOUR ANSWER HERE.
